In [1]:
import csv

import math
import numpy as np
import pandas as pd

import psycopg2
from geographiclib.geodesic import Geodesic

In [2]:
def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

In [5]:
def my_read_csv_file(file_name, limit):
    "read the csv file and print only the first limit rows"
    
    csv_file = open(file_name, "r")
    
    csv_data = csv.reader(csv_file)
    
    i = 0
    
    for row in csv_data:
        i += 1
        if i <= limit:
            print(row)
            
    print("\nPrinted ", min(limit, i), "lines of ", i, "total lines.")

## Drop the tables if they exist

In [6]:
connection.rollback()

query = """

drop table if exists stations;

"""

cursor.execute(query)

connection.commit()

connection.rollback()



In [7]:
query = """

drop table if exists lines;
"""

cursor.execute(query)

connection.commit()

connection.rollback()



In [8]:
query = """

drop table if exists travel_times;

"""

cursor.execute(query)

connection.commit()

## Create the tables

In [9]:
connection.rollback()

query = """

Create table stations (
    station varchar(32),
    latitude numeric(9,6),
    longitude numeric(9,6),
    transfer_time numeric(3),
    primary key (station));

"""

cursor.execute(query)

connection.commit()

In [10]:
connection.rollback()

query = """

Create table lines(
    line varchar(6),
    sequence numeric(2),
    station varchar(32),
    primary key (line, sequence));

"""

cursor.execute(query)

connection.commit()

In [11]:
connection.rollback()

query = """

Create table travel_times (
    station_1 varchar(32),
    station_2 varchar(32),
    travel_time numeric(3),
    primary key (station_1, station_2));

"""

cursor.execute(query)

connection.commit()

## Load data into database table

In [12]:
connection.rollback()

query = """

copy stations
from '/user/projects/project-3-sallyfang00/code/stations.csv' delimiter ',' NULL '' csv header;

"""

cursor.execute(query)

connection.commit()

In [13]:
connection.rollback()

query = """

copy lines
from '/user/projects/project-3-sallyfang00/code/lines.csv' delimiter ',' NULL '' csv header;


"""

cursor.execute(query)

connection.commit()

In [14]:
connection.rollback()

query = """

copy travel_times
from '/user/projects/project-3-sallyfang00/code/travel_times.csv' delimiter ',' NULL '' csv header;


"""

cursor.execute(query)

connection.commit()

## Query the tables needed

In [15]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT station
FROM stations

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station
0,12th Street
1,16th Street Mission
2,19th Street
3,24th Street Mission
4,Antioch
5,Ashby
6,Balboa Park
7,Bay Fair
8,Berryessa
9,Castro Valley


In [16]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT station, line
FROM lines
ORDER BY station, line;

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,line
0,12th Street,orange
1,12th Street,red
2,12th Street,yellow
3,16th Street Mission,blue
4,16th Street Mission,green
...,...,...
109,West Dublin,blue
110,West Oakland,blue
111,West Oakland,green
112,West Oakland,red


In [17]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT a.station, a.line AS from_line, b.line AS to_line, c.transfer_time
FROM lines a JOIN lines b ON a.station = b.station AND a.line != b.line JOIN stations c ON a.station = c.station
ORDER BY station, from_line, to_line;

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,station,from_line,to_line,transfer_time
0,12th Street,orange,red,282
1,12th Street,orange,yellow,282
2,12th Street,red,orange,282
3,12th Street,red,yellow,282
4,12th Street,yellow,orange,282
...,...,...,...,...
203,West Oakland,red,green,283
204,West Oakland,red,yellow,283
205,West Oakland,yellow,blue,283
206,West Oakland,yellow,green,283


In [18]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT a.line, a.station AS from_station, b.station AS to_station, c.travel_time
FROM lines a JOIN lines b ON a.line = b.line AND a.sequence+1 = b.sequence JOIN travel_times C ON (a.station = c.station_1 AND b.station = c.station_2) OR (b.station = c.station_1 AND a.station = c.station_2) 
ORDER BY line, from_station, to_station;

"""

my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)

,line,from_station,to_station,travel_time
0,blue,16th Street Mission,24th Street Mission,120
1,blue,24th Street Mission,Glen Park,180
2,blue,Balboa Park,Daly City,240
3,blue,Bay Fair,San Leandro,240
4,blue,Castro Valley,Bay Fair,240
...,...,...,...,...
103,yellow,Rockridge,MacArthur,240
104,yellow,San Bruno,SFO,240
105,yellow,South San Francisco,San Bruno,240
106,yellow,Walnut Creek,Lafayette,300


## Create a graph database in Neo4j for the BART system

In [19]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

In [20]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [21]:
session = driver.session(database="neo4j")

In [22]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [23]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [24]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [25]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [26]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [27]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [28]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [29]:
cursor = connection.cursor()

In [30]:
my_neo4j_wipe_out_database()

In [31]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


In [32]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

In [33]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


In [34]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [35]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


In [36]:
connection.rollback()

query = """



SELECT a.station, a.line AS from_line, b.line AS to_line, c.transfer_time
FROM lines a JOIN lines b ON a.station = b.station AND a.line != b.line JOIN stations c ON a.station = c.station
ORDER BY station, from_line, to_line;



"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)
    

In [37]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


In [38]:
connection.rollback()

query = """

SELECT a.line, a.station AS from_station, b.station AS to_station, c.travel_time
FROM lines a JOIN lines b ON a.line = b.line AND a.sequence+1 = b.sequence JOIN travel_times C ON (a.station = c.station_1 AND b.station = c.station_2) OR (b.station = c.station_1 AND a.station = c.station_2) 
ORDER BY line, from_station, to_station;

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [39]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


## Implement Neo4j Algorithms 

## 1: Dijkatra Shortest Path

In [40]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

In [41]:
my_neo4j_shortest_path('depart Ashby','arrive Hayward')


--------------------------------
   Total Cost:  1980
   Minutes:  33.0
--------------------------------
depart Ashby, 0, 0
orange Ashby, 0, 0
orange MacArthur, 240, 240
orange 19th Street, 180, 420
orange 12th Street, 120, 540
orange Lake Merritt, 180, 720
orange Fruitvale, 300, 1020
orange Coliseum, 240, 1260
orange San Leandro, 240, 1500
orange Bay Fair, 240, 1740
orange Hayward, 240, 1980
arrive Hayward, 0, 1980


## 2: Weighted Betweenness Centrality 


In [42]:
def my_calculate_box(point, miles):
    "Given a point and miles, calculate the box in form left, right, top, bottom"
    
    geod = Geodesic.WGS84

    kilometers = miles * 1.60934
    meters = kilometers * 1000

    g = geod.Direct(point[0], point[1], 270, meters)
    left = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 90, meters)
    right = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 0, meters)
    top = (g['lat2'], g['lon2'])

    g = geod.Direct(point[0], point[1], 180, meters)
    bottom = (g['lat2'], g['lon2'])
    
    return(left, right, top, bottom)

In [43]:
def my_station_get_population(station, miles):
    "given a station, pull all zip codes with miles distance, print them, sum the population"
    
    connection.rollback()
    
    query = "select latitude, longitude from stations "
    query += "where station = '" + station + "'"
    
    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
    for row in rows:
        latitude = row[0]
        longitude = row[1]
        
    point = (latitude, longitude)
        
    (left, right, top, bottom) = my_calculate_box(point, miles)
    
    query = "select zip, population from zip_codes "
    query += " where latitude >= " + str(bottom[0])
    query += " and latitude <= " + str(top [0])
    query += " and longitude >= " + str(left[1])
    query += " and longitude <= " + str(right[1])
    query += " order by 1 "

    cursor.execute(query)
    
    connection.rollback()
    
    rows = cursor.fetchall()
    
#     print("\n-------------------------------------------------------------------------------")
#     print(station + " BART Station")
#     print("-------------------------------------------------------------------------------\n")
    
    total_population = 0
    
    for row in rows:
        zip = row[0]
        population = row[1]
        #print("     zip:", zip, "  population: ", f'{population:10,}')
        total_population += population
        
    
#     print("\n-------------------------------------------------------------------------------")
#     print("  Total Population: ", f'{total_population:10,}')
#     print("-------------------------------------------------------------------------------")
    return total_population

In [44]:
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

query = """

CALL gds.betweenness.stream('ds_graph')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
ORDER BY betweenness DESC

"""

df = my_neo4j_run_query_pandas(query)

In [45]:
df = df[~df['name'].astype(str).str.startswith('arrive')]
df = df[~df['name'].astype(str).str.startswith('depart')]
df['station'] = df['name'].astype(str).str.split(" ")
df['station'] = df['station'].str[1:]
df['station'] = df['station'].apply(lambda x: ' '.join(map(str, x)))
df2 = df.groupby('station').mean('betweenness').reset_index().sort_values(by = 'betweenness', ascending = False)
df2 = df2.rename(columns = {'betweenness': 'average_betweenness'})
df2

,station,average_betweenness
39,Rockridge,5509.000000
33,Orinda,4997.000000
24,Lafayette,4469.000000
0,12th Street,4176.426217
26,MacArthur,4138.430736
25,Lake Merritt,3988.568790
46,Walnut Creek,3925.000000
2,19th Street,3888.013767
21,Fruitvale,3697.419834
49,West Oakland,3476.772638


In [46]:
pop = []
for station in df2['station'].values:
    pop.append(my_station_get_population(station, 3))
df_pop = pd.DataFrame({'station': df2['station'].values, 'population_3miles': pop})
df_pop = df_pop.sort_values(by = 'population_3miles', ascending = False)
df_pop

,station,population_3miles
28,Glen Park,624151
24,24th Street Mission,621661
22,16th Street Mission,554106
33,Balboa Park,548413
19,Civic Center,500384
15,Powell Street,496390
11,Embarcadero,425020
12,Montgomery Street,425020
27,Colma,341124
35,Daly City,330204


In [47]:
from decimal import Decimal
#weighing the average betweenness per station 

df_weighted = pd.merge(df2, df_pop, on = 'station')
total_pop = float(np.sum(df_weighted['population_3miles']))
df_weighted['population_3miles'] = df_weighted['population_3miles'].apply(lambda x: float(x))
df_weighted['weighted_average_betweenness'] = df_weighted['average_betweenness']*df_weighted['population_3miles']/total_pop

df_weighted.sort_values(by = 'weighted_average_betweenness', ascending = False)

,station,average_betweenness,population_3miles,weighted_average_betweenness
0,Rockridge,5509.000000,317026.0,144.121016
15,Powell Street,2825.404823,496390.0,115.734641
11,Embarcadero,3207.216667,425020.0,112.485704
24,24th Street Mission,2118.130725,621661.0,108.659085
22,16th Street Mission,2376.078095,554106.0,108.645871
19,Civic Center,2611.338944,500384.0,107.826702
12,Montgomery Street,3021.401362,425020.0,105.968662
3,12th Street,4176.426217,298398.0,102.839601
8,Fruitvale,3697.419834,317296.0,96.810625
7,19th Street,3888.013767,298398.0,95.737783


## 3: Community Detection (Louvain Modularity)

In [48]:
#community label from LPA
query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

query = """

CALL gds.labelPropagation.stream('ds_graph')
YIELD nodeId, communityId AS Community
RETURN gds.util.asNode(nodeId).name AS Name, Community
ORDER BY Community, Name

"""


my_neo4j_run_query_pandas(query)


,Name,Community
0,depart Downtown Berkeley,1
1,depart El Cerrito Plaza,1
2,depart El Cerrito del Norte,1
3,depart North Berkeley,1
4,depart Richmond,1
...,...,...
209,arrive Union City,266
210,arrive Walnut Creek,268
211,arrive Warm Springs,270
212,arrive West Dublin,272


In [49]:
#Louvain Modularity

query = "CALL gds.graph.drop('ds_graph', false)"
session.run(query)

query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
session.run(query)

query = """

CALL gds.louvain.stream('ds_graph')
YIELD nodeId, communityId, intermediateCommunityIds
RETURN gds.util.asNode(nodeId).name AS name, communityId as community, intermediateCommunityIds as intermediate_community
ORDER BY community, name ASC

"""

df_co = my_neo4j_run_query_pandas(query)


In [50]:
df_co = df_co[~df_co['name'].astype(str).str.startswith('arrive')]
df_co = df_co[~df_co['name'].astype(str).str.startswith('depart')]
df_co['station'] = df_co['name'].astype(str).str.split(" ")
df_co['station'] = df_co['station'].str[1:]
df_co['station'] = df_co['station'].apply(lambda x: ' '.join(map(str, x)))
df_co1 = df_co.groupby('station').mean('community').reset_index().sort_values(by = 'community', ascending = True)
df_co1 = df_co1.rename(columns = {'community': 'average_community'})
df_co1

,station,average_community
0,12th Street,74.0
2,19th Street,74.0
26,MacArthur,74.0
1,16th Street Mission,76.0
3,24th Street Mission,76.0
6,Balboa Park,82.0
22,Glen Park,82.0
14,Daly City,82.0
28,Milpitas,86.0
20,Fremont,86.0


In [51]:
df_co1.average_community.value_counts()

162.0    10
102.0     7
106.0     6
86.0      5
150.0     5
120.0     4
74.0      3
82.0      3
128.0     3
76.0      2
168.0     2
Name: average_community, dtype: int64